In [55]:
import pandas as pd

In [56]:
# from google.colab import drive
# drive.mount('/content/drive')

In [57]:
'''import os
os.listdir('/content/drive/My Drive/')'''

"import os\nos.listdir('/content/drive/My Drive/')"

data problem
1. read csv directly
2. combine data

In [58]:
df_train_csv = pd.read_csv('train.csv')
df_test_csv = pd.read_csv('test.csv')

In [59]:
df_train_csv['sii']
# df_test_csv.head()

0       2.0
1       0.0
2       0.0
3       1.0
4       NaN
       ... 
3955    1.0
3956    NaN
3957    1.0
3958    0.0
3959    NaN
Name: sii, Length: 3960, dtype: float64

In [60]:
df_train_csv_filter = df_train_csv[df_train_csv['sii'].notna()]

In [61]:
df_train_csv_filter['sii']

0       2.0
1       0.0
2       0.0
3       1.0
5       1.0
       ... 
3953    0.0
3954    1.0
3955    1.0
3957    1.0
3958    0.0
Name: sii, Length: 2736, dtype: float64

In [62]:
# Make sure features are same at train dataset and test dataset
# common_columns = list(set(df_train_csv_filter.columns) & set(df_test_csv.columns))

# df_train_csv_filter = df_train_csv_filter[sorted(common_columns)]
# df_test_csv = df_test_csv[sorted(common_columns)]

# All versions
1. all features
2. all features + area weight
3. all features + multicolinear(VIF)
4. all features + multicolinear(VIF) + area weight
5. all features + dimension reduce(PCA)
6. all features + dimension reduce(PCA) + area weight
7. all features + multicolinear(VIF) + dimension reduce(PCA)
8. all features + multicolinear(VIF) + dimension reduce(PCA) + area weight


# All Features
1. feature aggregation ->
  1. BMI, height, weight -> use Physical-BMI directly; delete height, weight columns
  2. convert time(mins)
2. season data -> whether need to delete[if not, categorical -> numberical]
  [default:delete directly]
3. missing value[if model can deal with missing value, do not deal with; if not, using mean to impute]
  1. prioritize with model can deal with missing values internally

In [63]:
# train

# feature aggregation -- BMI, height, weight
df_train_csv_filter = df_train_csv_filter.drop(columns=['BIA-BIA_BMI', 'Physical-Weight', 'Physical-Height'])

# convert time (mins)
df_train_csv_filter['Fitness_Endurance-Time'] = df_train_csv_filter['Fitness_Endurance-Time_Mins'] + df_train_csv_filter['Fitness_Endurance-Time_Sec']/60
df_train_csv_filter = df_train_csv_filter.drop(columns=['Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec'])

# default --- delete all season columns
season_columns = [col for col in df_train_csv_filter.columns if "Season" in col]
df_train_csv_filter = df_train_csv_filter.drop(columns=season_columns)


In [64]:
# test

# feature aggregation -- BMI, height, weight
df_test_csv = df_test_csv.drop(columns=['BIA-BIA_BMI', 'Physical-Weight', 'Physical-Height'])

# convert time (mins)
df_test_csv['Fitness_Endurance-Time'] = df_test_csv['Fitness_Endurance-Time_Mins'] + df_test_csv['Fitness_Endurance-Time_Sec']/60
df_test_csv = df_test_csv.drop(columns=['Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec'])

# default --- delete all season columns
season_columns = [col for col in df_test_csv.columns if "Season" in col]
df_test_csv = df_test_csv.drop(columns=season_columns)

In [65]:
train_features=pd.read_csv('train_features.csv')
trian_stat=pd.read_csv('train_stat.csv')
train_time_series=pd.read_csv('train_time_series_features.csv')
test_features=pd.read_csv('test_features.csv')
test_stat=pd.read_csv('test_stat.csv')
test_time_series=pd.read_csv('test_time_series_features.csv')

In [66]:
df_train_raw_features=df_train_csv_filter.merge(train_features,on='id')
df_train_raw_stat=df_train_csv_filter.merge(trian_stat,on='id')
df_train_raw_time_series=df_train_csv_filter.merge(train_time_series,on='id')
df_test_raw_features=df_test_csv.merge(test_features,on='id')
df_test_raw_stat=df_test_csv.merge(test_stat,on='id')
df_test_raw_time_series=df_test_csv.merge(test_time_series,on='id')

In [67]:
df_train_raw_features.to_csv('df_train_raw_features.csv')
df_train_raw_stat.to_csv('df_train_raw_stat.csv')
df_train_raw_time_series.to_csv('df_train_raw_time_series.csv')
df_test_raw_features.to_csv('df_test_raw_features.csv')
df_test_raw_stat.to_csv('df_test_raw_stat.csv')
df_test_raw_time_series.to_csv('df_test_raw_time_series.csv')

In [68]:
'''
# not delete season data, categorical -> numberical
season_mapping = {"Spring": 1, "Summer": 2, "Fall": 3, "Winter": 4, "NaN": 5}

season_columns = [col for col in df_train_csv_filter.columns if "Season" in col]

for col in season_columns:
    df_train_csv_filter.loc[:, col] = df_train_csv_filter[col].map(
        lambda x: 5 if pd.isna(x) else season_mapping.get(x, x)
    )
df_train_csv_filter
'''

'\n# not delete season data, categorical -> numberical\nseason_mapping = {"Spring": 1, "Summer": 2, "Fall": 3, "Winter": 4, "NaN": 5}\n\nseason_columns = [col for col in df_train_csv_filter.columns if "Season" in col]\n\nfor col in season_columns:\n    df_train_csv_filter.loc[:, col] = df_train_csv_filter[col].map(\n        lambda x: 5 if pd.isna(x) else season_mapping.get(x, x)\n    )\ndf_train_csv_filter\n'

# Multicolinear(VIF)

In [69]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

# X = df_train_csv_filter.drop(columns=['sii']).dropna()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

print(vif_data)

NameError: name 'X' is not defined

# Dimension Reduce(PCA)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df_train_csv_filter.drop(columns=['sii']).dropna()
y = df_train_csv_filter['sii']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_scaled)

explained_variance = pca.explained_variance_ratio_
print(f"Explained variance by each component: {explained_variance}")
print(f"Total number of components selected: {pca.n_components_}")

X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])
X_pca_df['sii'] = y.reset_index(drop=True)

print(X_pca_df.head())